# Visualizzazione Plate SRT

Questo notebook visualizza i backgrounds estratti da osservazioni SRT reali.
Questi plate vengono usati per iniettare segnali durante il training.

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
import os

from src.data.cadence_generator import CadenceGenerator, CadenceParams
from src.utils.preprocessing import normalize_log

plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12

## 1. Caricamento Plate SRT

In [ ]:
PLATE_PATH = '../data/srt_training/srt_backgrounds.npz'

if os.path.exists(PLATE_PATH):
    plate_data = np.load(PLATE_PATH)
    plates = plate_data['backgrounds']
    print(f'Shape: {plates.shape}')
    print(f'N backgrounds: {len(plates)}')
    print(f'Memory: {plates.nbytes / 1e9:.2f} GB')
else:
    print(f'Plate not found: {PLATE_PATH}')
    plates = None

## 2. Backgrounds Casuali

In [ ]:
if plates is not None:
    n_samples = 4
    indices = np.random.choice(len(plates), n_samples, replace=False)
    
    fig, axes = plt.subplots(n_samples, 6, figsize=(18, 3*n_samples))
    labels = ['A1 (ON)', 'B (OFF)', 'A2 (ON)', 'C (OFF)', 'A3 (ON)', 'D (OFF)']
    
    for row, idx in enumerate(indices):
        bg = plates[idx]
        for col in range(6):
            obs = bg[col].reshape(16, 512, 8).mean(axis=2)
            axes[row, col].imshow(normalize_log(obs), aspect='auto', cmap='hot', vmin=0, vmax=1)
            if row == 0:
                axes[row, col].set_title(labels[col])
            if col == 0:
                axes[row, col].set_ylabel(f'BG {idx}')
            axes[row, col].set_xticks([])
            axes[row, col].set_yticks([])
    
    plt.suptitle('SRT Real Backgrounds', fontsize=14, color='orange')
    plt.tight_layout()
    plt.show()

## 3. Confronto: Sintetico vs Reale

In [ ]:
if plates is not None:
    fig, axes = plt.subplots(2, 6, figsize=(18, 6))
    labels = ['A1 (ON)', 'B (OFF)', 'A2 (ON)', 'C (OFF)', 'A3 (ON)', 'D (OFF)']
    
    # Synthetic
    synth_gen = CadenceGenerator(CadenceParams(fchans=512, tchans=16), seed=42)
    synth_bg = synth_gen._get_background()
    for col in range(6):
        axes[0, col].imshow(normalize_log(synth_bg[col]), aspect='auto', cmap='hot', vmin=0, vmax=1)
        axes[0, col].set_title(labels[col])
        axes[0, col].set_xticks([])
        axes[0, col].set_yticks([])
    axes[0, 0].set_ylabel('Synthetic', fontsize=12)
    
    # Real
    real_bg = plates[np.random.randint(len(plates))]
    for col in range(6):
        obs = real_bg[col].reshape(16, 512, 8).mean(axis=2)
        axes[1, col].imshow(normalize_log(obs), aspect='auto', cmap='hot', vmin=0, vmax=1)
        axes[1, col].set_xticks([])
        axes[1, col].set_yticks([])
    axes[1, 0].set_ylabel('SRT Real', fontsize=12)
    
    plt.suptitle('Synthetic vs Real Noise', fontsize=14)
    plt.tight_layout()
    plt.show()

## 4. TRUE Sample con Background SRT

In [ ]:
if plates is not None:
    srt_gen = CadenceGenerator(
        CadenceParams(fchans=4096, tchans=16, snr_base=30, snr_range=20),
        plate=plates, seed=42
    )
    true_srt = srt_gen.create_true_sample_fast()
    
    fig, axes = plt.subplots(2, 6, figsize=(18, 6))
    labels = ['A1 (ON)', 'B (OFF)', 'A2 (ON)', 'C (OFF)', 'A3 (ON)', 'D (OFF)']
    
    for col in range(6):
        obs = true_srt[col].reshape(16, 512, 8).mean(axis=2)
        axes[0, col].imshow(obs, aspect='auto', cmap='hot')
        axes[0, col].set_title(labels[col])
        axes[0, col].set_xticks([])
        axes[0, col].set_yticks([])
        
        axes[1, col].imshow(normalize_log(obs), aspect='auto', cmap='hot', vmin=0, vmax=1)
        axes[1, col].set_xticks([])
        axes[1, col].set_yticks([])
    
    axes[0, 0].set_ylabel('Raw')
    axes[1, 0].set_ylabel('Normalized')
    plt.suptitle('TRUE Sample + Real SRT Background', fontsize=14, color='lime')
    plt.tight_layout()
    plt.show()

## 5. FALSE Sample con Background SRT

In [ ]:
if plates is not None:
    false_srt = srt_gen.create_false_sample()
    
    fig, axes = plt.subplots(2, 6, figsize=(18, 6))
    
    for col in range(6):
        obs = false_srt[col].reshape(16, 512, 8).mean(axis=2)
        axes[0, col].imshow(obs, aspect='auto', cmap='hot')
        axes[0, col].set_title(labels[col])
        axes[0, col].set_xticks([])
        axes[0, col].set_yticks([])
        
        axes[1, col].imshow(normalize_log(obs), aspect='auto', cmap='hot', vmin=0, vmax=1)
        axes[1, col].set_xticks([])
        axes[1, col].set_yticks([])
    
    axes[0, 0].set_ylabel('Raw')
    axes[1, 0].set_ylabel('Normalized')
    plt.suptitle('FALSE Sample + Real SRT Background', fontsize=14, color='red')
    plt.tight_layout()
    plt.show()

## 6. Statistiche Plate

In [ ]:
if plates is not None:
    sample = plates[:min(100, len(plates))]
    
    print(f'Mean power: {np.mean(sample):.2e}')
    print(f'Std power:  {np.std(sample):.2e}')
    print(f'Min:        {np.min(sample):.2e}')
    print(f'Max:        {np.max(sample):.2e}')
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    axes[0].hist(sample[:10].flatten(), bins=100, color='orange', alpha=0.7)
    axes[0].set_xlabel('Power')
    axes[0].set_ylabel('Count')
    axes[0].set_title('Power Distribution')
    axes[0].set_yscale('log')
    
    var_per_obs = [np.var(obs) for bg in sample for obs in bg]
    axes[1].hist(var_per_obs, bins=50, color='cyan', alpha=0.7)
    axes[1].set_xlabel('Variance')
    axes[1].set_ylabel('Count')
    axes[1].set_title('Observation Variance')
    
    plt.tight_layout()
    plt.show()